In [11]:
import sqlite3

from helpers.constants import QUERY

In [10]:
# Forcing module reload

import importlib
import helpers.constants
importlib.reload(helpers.constants)

<module 'helpers.constants' from 'd:\\Documents\\GitHub\\Pivot_Point_Trading_Bot\\helpers\\constants.py'>

In [19]:
conn = sqlite3.connect("Futures.db")
cursor = conn.cursor()

In [20]:
cursor.execute(QUERY)
x = cursor.fetchall()
conn.close()

In [21]:
high = x[0][0]
low = x[0][1]
close = x[0][2]

print(close)

5771.75


In [ ]:
# Getting all tables from sqlite_master
sql_query = """SELECT name FROM sqlite_master 
WHERE type='table';"""

# Creating cursor object using connection object
cursor = conn.cursor()

# executing our sql query
cursor.execute(sql_query)
print("List of tables\n")

# printing all tables list
print(cursor.fetchall())

List of tables

[('ES_1m',), ('ES_5m',), ('ES_15m',), ('ES_30m',), ('ES_1h',), ('YM_1m',), ('YM_5m',), ('YM_15m',), ('YM_30m',), ('YM_1h',), ('NQ_1m',), ('NQ_5m',), ('NQ_15m',), ('NQ_30m',), ('NQ_1h',), ('RT_1m',), ('RT_5m',), ('RT_15m',), ('RT_30m',), ('RT_1h',), ('CL_1m',), ('CL_5m',), ('CL_15m',), ('CL_30m',), ('CL_1h',), ('GC_1m',), ('GC_5m',), ('GC_15m',), ('GC_30m',), ('GC_1h',), ('SI_1m',), ('SI_5m',), ('SI_15m',), ('SI_30m',), ('SI_1h',), ('VIX_1m',), ('VIX_5m',), ('VIX_15m',), ('VIX_30m',), ('VIX_1h',)]


In [ ]:
import pandas as pd
import yfinance as yf
from datetime import timedelta

# Calculate start date
today = pd.to_datetime("today").normalize()
# Only counts business days
start = today - pd.tseries.offsets.BDay(5)

# Downloading correct data from yfinance
df = yf.download("^GSPC", start=start, interval="15m", prepost=False)
df = df.xs("^GSPC", axis=1, level="Ticker")
df.index = df.index.tz_convert("America/Chicago")
df.head(5)

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Datetime,,,,,
2025-04-16 08:30:00-05:00,5343.870117,5355.189941,5331.990234,5335.750000,89887303
2025-04-16 08:45:00-05:00,5316.859863,5343.870117,5316.859863,5343.870117,123457695
2025-04-16 09:00:00-05:00,5320.229980,5332.529785,5314.500000,5316.229980,121879612
2025-04-16 09:15:00-05:00,5335.479980,5338.850098,5317.850098,5319.959961,100686287
2025-04-16 09:30:00-05:00,5344.729980,5346.270020,5331.649902,5335.439941,91724536


In [ ]:
# Split Datetime into Date, Time
df['Date'] = df.index.strftime('%Y-%m-%d')  # Extract Date
df['Time'] = df.index.strftime('%H:%M:%S')  # Extract Time without timezone
df.tail()

# Drop -05:00
df = df.reset_index(drop=True)  # Remove original Datetime index if needed
df.tail()

# Delete all rows with Time between 15:15 & 08:15
# Convert Time column to datetime.time for proper comparison
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time

# Define the start and end time boundaries
start_time = pd.to_datetime("15:15:00", format='%H:%M:%S').time()
end_time = pd.to_datetime("08:15:00", format='%H:%M:%S').time()

# Keep only rows where Time is NOT between 15:15:00 and 08:15:00
df = df[~((df['Time'] >= start_time) | (df['Time'] <= end_time))]

# Reset index after filtering (optional)
df = df.reset_index(drop=True)
df.tail()

# Retrieve Max value for High column

# Retrieve Min value for Low column

# Retrieve most recent value for Close column

# Perform calculations

Price,Close,High,Low,Open,Volume,Date,Time
99,5284.379883,5288.270020,5277.569824,5277.569824,70561000,2025-04-22,13:45:00
100,5273.680176,5287.169922,5272.580078,5284.259766,68341000,2025-04-22,14:00:00
101,5263.319824,5275.660156,5259.810059,5273.750000,86745000,2025-04-22,14:15:00
102,5278.810059,5281.359863,5259.100098,5263.140137,121346000,2025-04-22,14:30:00
103,5286.810059,5291.810059,5274.069824,5278.470215,327807000,2025-04-22,14:45:00


In [7]:
df.head()

Price,Close,High,Low,Open,Volume,Date,Time
0,5343.870117,5355.189941,5331.990234,5335.750000,89887303,2025-04-16,08:30:00
1,5316.859863,5343.870117,5316.859863,5343.870117,123457695,2025-04-16,08:45:00
2,5320.229980,5332.529785,5314.500000,5316.229980,121879612,2025-04-16,09:00:00
3,5335.479980,5338.850098,5317.850098,5319.959961,100686287,2025-04-16,09:15:00
4,5344.729980,5346.270020,5331.649902,5335.439941,91724536,2025-04-16,09:30:00


In [ ]:
max_high = df['High'].max()  # Maximum value in High column
min_low = df['Low'].min()    # Minimum value in Low column
latest_close = df['Close'].iloc[-1]  # Most recent Close value

print(
    f"Max High: {max_high}, Min Low: {min_low}, Latest Close: {latest_close}")

Max High: 5367.240234375, Min Low: 5101.6298828125, Latest Close: 5286.81005859375


In [ ]:
# Calculations

ran = max_high - min_low
print(ran)
pp = round((max_high + min_low + latest_close) / 3, 2)
print(pp)
r1 = round(2 * pp - min_low, 2)
print(r1)
r2 = round(pp + ran, 2)
print(r2)
r3 = round(pp + ran * 2, 2)
print(r3)
s1 = round(2 * pp - max_high, 2)
print(s1)
s2 = round(pp - ran, 2)
print(s2)
s3 = round(pp - ran * 2, 2)
print(s3)

265.6103515625
5251.89
5402.15
5517.5
5783.11
5136.54
4986.28
4720.67


In [51]:
# Downloading Data
curr_price = yf.download(
    "ES=F", start=pd.to_datetime("today") - timedelta(1))

# Reformatting Multi-Index
curr_price = curr_price.xs('ES=F', axis=1, level='Ticker')

# # Converting Times
# curr_price.index = curr_price.index.tz_convert(
#     'America/Chicago')

# Retrieving Last Price
last = curr_price[['Close']].iloc[-1]
last = last.iloc[0]
print(type(last))

# logger.info(f"Curr. Price: {last}\n")

# Pulling Last Row
curr_price = curr_price.iloc[-1]

[*********************100%***********************]  1 of 1 completed

<class 'numpy.float64'>


In [52]:
ticker = "ES=F"

data = yf.Ticker(ticker)
price = data.history(period="1d", interval="1m").iloc[-1]["Close"]
print(price)

5690.0


In [55]:
curr_price = yf.download(
    "ES=F", start=pd.to_datetime("today") - timedelta(1), interval='15m')

# Reformatting Multi-Index
curr_price = curr_price.xs('ES=F', axis=1, level='Ticker')

# # Converting Times
curr_price.index = curr_price.index.tz_convert(
    'America/Chicago')
print(curr_price)

[*********************100%***********************]  1 of 1 completed

Price                        Close     High      Low     Open  Volume
Datetime                                                             
2025-03-09 17:00:00-05:00  5729.25  5756.25  5726.75  5753.00   11505
2025-03-09 17:15:00-05:00  5733.50  5738.75  5722.75  5729.00    9280
2025-03-09 17:30:00-05:00  5738.00  5741.25  5730.75  5733.50    4375
2025-03-09 17:45:00-05:00  5728.00  5741.50  5727.50  5738.25    3555
2025-03-09 18:00:00-05:00  5728.50  5732.00  5710.50  5728.00   11066
...                            ...      ...      ...      ...     ...
2025-03-10 08:00:00-05:00  5699.50  5706.75  5691.25  5701.25   23605
2025-03-10 08:15:00-05:00  5691.50  5701.00  5690.25  5699.50   18350
2025-03-10 08:30:00-05:00  5707.50  5711.25  5679.25  5691.50  135808
2025-03-10 08:45:00-05:00  5689.25  5710.50  5682.25  5707.50  107442
2025-03-10 09:00:00-05:00  5684.50  5689.50  5678.50  5689.50   12783

[65 rows x 5 columns]


In [ ]:
import yfinance as yf
import time

ticker = yf.Ticker("ES=F")

while True:
    print(f"Latest Price: {ticker.fast_info['last_price']}")
    time.sleep(5)  # Adjust as needed

Latest Price: 5673.0
Latest Price: 5673.0
Latest Price: 5673.0
Latest Price: 5673.0


KeyboardInterrupt: 